In [39]:
import nltk
import sklearn
import pycrfsuite
from itertools import chain
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer




ModuleNotFoundError: No module named 'pandas'

In [2]:
nltk.download('conll2002')
nltk.corpus.conll2002.fileids()


[nltk_data] Downloading package conll2002 to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']

In [3]:
print(nltk.corpus.conll2002.iob_sents('esp.train'))
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

[[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')], [('-', 'Fg', 'O')], ...]


In [4]:
train_sents[0]

[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

In [5]:
#présenter données autrement
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'postag[:2]=' + postag[:2],
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            '-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            '+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [6]:
lst = [1, 3, 2, 9, 8]

In [7]:
lst2 = ['DAvid', 'Guillaume', 'PAris']

In [8]:
print(lst[:-1])

[1, 3, 2, 9]


In [9]:
print(lst[-1:])

[8]


In [10]:
print(lst[:2])

[1, 3]


In [11]:
print(lst[-2:])

[9, 8]


In [12]:
print(lst[2:])

[2, 9, 8]


In [13]:
print(lst[1:])

[3, 2, 9, 8]


In [14]:
sent2features(train_sents[0])[0]

['bias',
 'word.lower=melbourne',
 'word[-3:]=rne',
 'word[-2:]=ne',
 'word.isupper=False',
 'word.istitle=True',
 'word.isdigit=False',
 'postag=NP',
 'postag[:2]=NP',
 'BOS',
 '+1:word.lower=(',
 '+1:word.istitle=False',
 '+1:word.isupper=False',
 '+1:postag=Fpa',
 '+1:postag[:2]=Fp']

In [15]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

In [16]:
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)


In [17]:

trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [18]:

trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [19]:
trainer.train('conll2002-esp.crfsuite')

In [20]:

!ls -lh ./conll2002-esp.crfsuite


'ls' n'est pas reconnu en tant que commande interne
ou externe, un programme ex‚cutable ou un fichier de commandes.


In [21]:
trainer.logparser.last_iteration

{'num': 50,
 'scores': {},
 'loss': 14807.577946,
 'feature_norm': 79.110017,
 'error_norm': 1262.912078,
 'active_features': 11346,
 'linesearch_trials': 1,
 'linesearch_step': 1.0,
 'time': 0.277}

In [22]:

print(len(trainer.logparser.iterations), trainer.logparser.iterations[-1])

50 {'num': 50, 'scores': {}, 'loss': 14807.577946, 'feature_norm': 79.110017, 'error_norm': 1262.912078, 'active_features': 11346, 'linesearch_trials': 1, 'linesearch_step': 1.0, 'time': 0.277}


In [23]:
tagger = pycrfsuite.Tagger()
tagger.open('conll2002-esp.crfsuite')


In [24]:
example_sent = test_sents[0]
print(' '.join(sent2tokens(example_sent)), end='\n\n')

print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2labels(example_sent)))

La Coruña , 23 may ( EFECOM ) .

Predicted: B-LOC I-LOC O O O O B-ORG O O
Correct:   B-LOC I-LOC O O O O B-ORG O O


In [25]:
def traduction_lambda(tag):
    tag = tag.split('-', 1)
    return tag[::-1]
    
print(traduction_lambda('B-PERS'))
    
    
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    print(tagset)
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    print(tagset)
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )
y_pred = [tagger.tag(xseq) for xseq in X_test]
print(bio_classification_report(y_test, y_pred))

['PERS', 'B']
{'I-MISC', 'I-ORG', 'B-LOC', 'B-MISC', 'I-LOC', 'B-ORG', 'I-PER', 'B-PER'}
['B-LOC', 'I-LOC', 'B-MISC', 'I-MISC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER']
              precision    recall  f1-score   support

       B-LOC       0.78      0.75      0.76      1084
       I-LOC       0.66      0.60      0.63       325
      B-MISC       0.69      0.47      0.56       339
      I-MISC       0.61      0.49      0.54       557
       B-ORG       0.79      0.81      0.80      1400
       I-ORG       0.80      0.79      0.80      1104
       B-PER       0.82      0.87      0.84       735
       I-PER       0.87      0.93      0.90       634

   micro avg       0.78      0.76      0.77      6178
   macro avg       0.75      0.71      0.73      6178
weighted avg       0.77      0.76      0.76      6178
 samples avg       0.09      0.09      0.09      6178



c:\users\david\pycharmprojects\crf_evaluation\venv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\david\pycharmprojects\crf_evaluation\venv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
lst = [1, 2, 3]

In [27]:
lst2 = [4, 5, 6]

In [28]:
print(zip(lst, lst2))

In [29]:
for row in zip(lst, lst2):
    print(row)

(1, 4)
(2, 5)
(3, 6)


In [30]:
from collections import Counter
info = tagger.info()

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(info.transitions).most_common(15))

print("\nTop unlikely transitions:")
print_transitions(Counter(info.transitions).most_common()[-15:])

Top likely transitions:
B-ORG  -> I-ORG   8.631963
I-ORG  -> I-ORG   7.833706
B-PER  -> I-PER   6.998706
B-LOC  -> I-LOC   6.913675
I-MISC -> I-MISC  6.129735
B-MISC -> I-MISC  5.538291
I-LOC  -> I-LOC   4.983567
I-PER  -> I-PER   3.748358
B-ORG  -> B-LOC   1.727090
B-PER  -> B-LOC   1.388267
B-LOC  -> B-LOC   1.240278
O      -> O       1.197929
O      -> B-ORG   1.097062
I-PER  -> B-LOC   1.083332
O      -> B-MISC  1.046113

Top unlikely transitions:
I-PER  -> B-ORG   -2.056130
I-LOC  -> I-ORG   -2.143940
B-ORG  -> I-MISC  -2.167501
I-PER  -> I-ORG   -2.369380
B-ORG  -> I-PER   -2.378110
I-MISC -> I-PER   -2.458788
B-LOC  -> I-PER   -2.516414
I-ORG  -> I-MISC  -2.571973
I-LOC  -> B-PER   -2.697791
I-LOC  -> I-PER   -3.065950
I-ORG  -> I-PER   -3.364434
O      -> I-PER   -7.322841
O      -> I-MISC  -7.648246
O      -> I-ORG   -8.024126
O      -> I-LOC   -8.333815


In [31]:
print("%-3s -> %-4s %0.6f" % ('guillaume', 'personne', 82.5))

guillaume -> personne 82.500000


In [32]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-6s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(info.state_features).most_common(20))

print("\nTop negative:")
print_state_features(Counter(info.state_features).most_common()[-20:])

Top positive:
8.886516 B-ORG  word.lower=efe-cantabria
8.743642 B-ORG  word.lower=psoe-progresistas
5.769032 B-LOC  -1:word.lower=cantabria
5.195429 I-LOC  -1:word.lower=calle
5.116821 O      word.lower=mayo
4.990871 O      -1:word.lower=día
4.910915 I-ORG  -1:word.lower=l
4.721572 B-MISC word.lower=diversia
4.676259 B-ORG  word.lower=telefónica
4.334354 B-ORG  word[-2:]=-e
4.149862 B-ORG  word.lower=amena
4.141370 B-ORG  word.lower=terra
3.942852 O      word.istitle=False
3.926397 B-ORG  word.lower=continente
3.924672 B-ORG  word.lower=acesa
3.888706 O      word.lower=euro
3.856445 B-PER  -1:word.lower=según
3.812373 B-MISC word.lower=exteriores
3.807582 I-MISC -1:word.lower=1.9
3.807098 B-MISC word.lower=sanidad

Top negative:
-1.965379 O      word.lower=fundación
-1.981541 O      -1:word.lower=británica
-2.118347 O      word.lower=061
-2.190653 B-PER  word[-3:]=nes
-2.226373 B-ORG  postag=SP
-2.226373 B-ORG  postag[:2]=SP
-2.260972 O      word[-3:]=uia
-2.384920 O      -1:word.lower

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [7]:
filename = r"C:\Users\david\OneDrive\Documents\Données Data Science\conll2003_2_bis\train.txt".encode(encoding='UTF-8',errors='strict')


In [11]:
import pandas as pd
import codecs


doc = codecs.open(filename, 'rU', 'UTF-8')
dataset = pd.read_csv(doc, sep='\t', nrows=1000000)

In [12]:
print(dataset)

          -DOCSTART- -X- -X- O
0            EU NNP B-NP B-ORG
1           rejects VBZ B-VP O
2        German JJ B-NP B-MISC
3               call NN I-NP O
4                 to TO B-VP O
...                        ...
204561         three CD I-NP O
204562   Swansea NN B-NP B-ORG
204563             1 CD I-NP O
204564  Lincoln NNP I-NP B-ORG
204565             2 CD I-NP O

[204566 rows x 1 columns]


In [13]:
filename2 = r"C:\Users\david\OneDrive\Documents\Données Data Science\conll2003_2_bis\valid.txt".encode(encoding='UTF-8',errors='strict')

In [16]:
doc = codecs.open(filename2, 'rU', 'UTF-8')
dataset2 = pd.read_csv(doc, sep='\t', nrows=1000000)

In [17]:
print(dataset2)

                -DOCSTART- -X- -X- O
0                 CRICKET NNP B-NP O
1                            - : O O
2      LEICESTERSHIRE NNP B-NP B-ORG
3                    TAKE NNP I-NP O
4                     OVER IN B-PP O
...                              ...
51572                        . . O O
51573                       -- : O O
51574           Dhaka NNP B-NP B-ORG
51575        Newsroom NNP I-NP I-ORG
51576         880-2-506363 CD I-NP O

[51577 rows x 1 columns]


In [18]:
dataset.append(dataset2)

,-DOCSTART- -X- -X- O
0,CRICKET NNP B-NP O
1,- : O O
2,LEICESTERSHIRE NNP B-NP B-ORG
3,TAKE NNP I-NP O
4,OVER IN B-PP O
...,...
51572,. . O O
51573,-- : O O
51574,Dhaka NNP B-NP B-ORG
51575,Newsroom NNP I-NP I-ORG


In [21]:
for row in dataset2:
    dataset.append(row, ignore_index = True)


TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

In [29]:
import nltk

train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))
print(train_sents[0])

[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]


In [30]:

print(train_sents[1])

[('-', 'Fg', 'O')]


In [31]:
print(train_sents[2])

[('El', 'DA', 'O'), ('Abogado', 'NC', 'B-PER'), ('General', 'AQ', 'I-PER'), ('del', 'SP', 'I-PER'), ('Estado', 'NC', 'I-PER'), (',', 'Fc', 'O'), ('Daryl', 'VMI', 'B-PER'), ('Williams', 'NC', 'I-PER'), (',', 'Fc', 'O'), ('subrayó', 'VMI', 'O'), ('hoy', 'RG', 'O'), ('la', 'DA', 'O'), ('necesidad', 'NC', 'O'), ('de', 'SP', 'O'), ('tomar', 'VMN', 'O'), ('medidas', 'NC', 'O'), ('para', 'SP', 'O'), ('proteger', 'VMN', 'O'), ('al', 'SP', 'O'), ('sistema', 'NC', 'O'), ('judicial', 'AQ', 'O'), ('australiano', 'AQ', 'O'), ('frente', 'RG', 'O'), ('a', 'SP', 'O'), ('una', 'DI', 'O'), ('página', 'NC', 'O'), ('de', 'SP', 'O'), ('internet', 'NC', 'O'), ('que', 'PR', 'O'), ('imposibilita', 'VMI', 'O'), ('el', 'DA', 'O'), ('cumplimiento', 'NC', 'O'), ('de', 'SP', 'O'), ('los', 'DA', 'O'), ('principios', 'NC', 'O'), ('básicos', 'AQ', 'O'), ('de', 'SP', 'O'), ('la', 'DA', 'O'), ('Ley', 'NC', 'B-MISC'), ('.', 'Fp', 'O')]


In [32]:
print(train_sents[3])

[('La', 'DA', 'O'), ('petición', 'NC', 'O'), ('del', 'SP', 'O'), ('Abogado', 'NC', 'B-PER'), ('General', 'AQ', 'I-PER'), ('tiene', 'VMI', 'O'), ('lugar', 'NC', 'O'), ('después', 'RG', 'O'), ('de', 'SP', 'O'), ('que', 'CS', 'O'), ('un', 'DI', 'O'), ('juez', 'NC', 'O'), ('del', 'SP', 'O'), ('Tribunal', 'NC', 'B-ORG'), ('Supremo', 'AQ', 'I-ORG'), ('del', 'SP', 'O'), ('estado', 'NC', 'O'), ('de', 'SP', 'O'), ('Victoria', 'NC', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), ('se', 'P0', 'O'), ('viera', 'VMS', 'O'), ('forzado', 'AQ', 'O'), ('a', 'SP', 'O'), ('disolver', 'VMN', 'O'), ('un', 'DI', 'O'), ('jurado', 'NC', 'O'), ('popular', 'AQ', 'O'), ('y', 'CC', 'O'), ('suspender', 'VMN', 'O'), ('el', 'DA', 'O'), ('proceso', 'NC', 'O'), ('ante', 'SP', 'O'), ('el', 'DA', 'O'), ('argumento', 'NC', 'O'), ('de', 'SP', 'O'), ('la', 'DA', 'O'), ('defensa', 'NC', 'O'), ('de', 'SP', 'O'), ('que', 'CS', 'O'), ('las', 'DA', 'O'), ('personas', 'NC', 'O'), ('que', 'PR', 'O'),

In [33]:
print(train_sents[4])

[('Esta', 'DD', 'O'), ('página', 'NC', 'O'), ('web', 'AQ', 'O'), ('lleva', 'VMI', 'O'), ('un', 'DI', 'O'), ('mes', 'NC', 'O'), ('de', 'SP', 'O'), ('existencia', 'NC', 'O'), (',', 'Fc', 'O'), ('tiempo', 'NC', 'O'), ('en', 'SP', 'O'), ('el', 'DA', 'O'), ('que', 'PR', 'O'), ('ha', 'VAI', 'O'), ('sido', 'VSP', 'O'), ('visitada', 'VMP', 'O'), ('en', 'SP', 'O'), ('más', 'RG', 'O'), ('de', 'SP', 'O'), ('un', 'DI', 'O'), ('millón', 'NC', 'O'), ('de', 'SP', 'O'), ('ocasiones', 'NC', 'O'), (',', 'Fc', 'O'), ('y', 'CC', 'O'), ('facilita', 'VMI', 'O'), ('información', 'NC', 'O'), ('sobre', 'SP', 'O'), ('miles', 'PN', 'O'), ('de', 'SP', 'O'), ('crímenes', 'NC', 'O'), ('y', 'CC', 'O'), ('criminales', 'VMM', 'O'), ('ya', 'RG', 'O'), ('enjuiciados', 'VMP', 'O'), ('o', 'CC', 'O'), ('aún', 'RG', 'O'), ('perseguidos', 'VMM', 'O'), (',', 'Fc', 'O'), ('datos', 'NC', 'O'), ('que', 'PR', 'O'), ('salen', 'VMI', 'O'), ('de', 'SP', 'O'), ('artículos', 'NC', 'O'), ('de', 'SP', 'O'), ('periódicos', 'NC', 'O'), ('

In [34]:
from helpers.DocumentHelper import DocumentsHelper


ModuleNotFoundError: No module named 'helpers'

In [35]:
DATA_DIRECTORY: str = 'data'
TEST_FILE_NAME: str = f'{DATA_DIRECTORY}/test.txt'
VALID_FILE_NAME: str = f'{DATA_DIRECTORY}/valid.txt'
SPACY_FILE_NAME: str = f'{DATA_DIRECTORY}/spacy.txt'

"""
    string management
"""
SPACE: str = ' '
EMPTY: str = ''
NEW_LINE: str = '\n'
OPEN_PARENTHESIS: str = '('
END_PARENTHESIS: str = ')'
DOT: str = '.'
COMMA: str = ','
COLONS: str = ':'
SLASH: str = '/'
QUOTE: str = "'"


"""
    document mnagament
"""
DOCUMENT_START: str = '-DOCSTART-'
LINE_START: str = '- : O O'
LINE_END: str = '. . O O'

"""
    data files columns
"""
INDEX_WORD: int = 0
INDEX_POST_TAG: int = 2
INDEX_ENTITY: int = 3

In [36]:
class DocumentsHelper:

    def remove_last_char(self, value: str) -> str:
        return value.rstrip(self.get_last_char(value))

    def get_last_char(self, value: str) -> str:
        if len(value) > 0:
            return value[-1]
        return ''

    def is_parenthesis(self, value: str) -> bool:
        return value == OPEN_PARENTHESIS or value == END_PARENTHESIS

    def is_special_char_without_left_space(self, value: str) -> bool:
        return value == COMMA or value == DOT or value == SLASH or value == QUOTE or value == END_PARENTHESIS

    def is_special_char_without_right_space(self, value: str) -> bool:
        return value == OPEN_PARENTHESIS or value == SLASH

    def clean_string(self, value: str) -> str:
        return value.replace(NEW_LINE, EMPTY).replace('\'', "'")

    def is_string_start_line(self, line: str) -> bool:
        return line.__contains__(LINE_START)

    def is_string_end_line(self, line: str) -> bool:
        return line.__contains__(LINE_END)

    def is_string_not_start_or_end_line(self, line: str) -> bool:
        return not (self.is_string_start_line(line) or self.is_string_end_line(line))

    def is_string_not_document_start(self, line: str) -> bool:
        return not line.__contains__(DOCUMENT_START)

    def is_string_not_empty_or_none(self, value: str) -> bool:
       return not (value is None or value == '')

    def is_valid_token(self, line: str) -> bool:
        return self.is_string_not_empty_or_none(line) and self.is_string_not_document_start(line) and self.is_string_not_start_or_end_line(line)

    def is_documents_property_empty(self, documents: list) -> bool:
        return self.get_documents_size(documents) == 0

    def get_documents_size(self, documents: list) -> int:
        return len(documents)

In [ ]:
def concatenate_data(filename_3):
    doc_lines = []
    helper = DocumentsHelper()
    file = open(filename_3, 'r')
    lines = file.readlines()
    for line in lines:
        #print(line)
        split_line = line.split(SPACE)
        #print(split_line)
        word: str = split_line[INDEX_WORD]
        try:
            if helper.is_string_start_line(line):
                doc_line = []
            if word != NEW_LINE and not helper.is_string_end_line(line) and not helper.is_string_start_line(line):
                entity: str = split_line[INDEX_ENTITY]
                post_tag: str = split_line[INDEX_POST_TAG]
                line_component = (word, post_tag, entity)
                doc_line.append(line_component)
            if helper.is_string_end_line(line):
                doc_lines.append(doc_line)
        except Exception as e:
            pass
    return doc_lines

concatenate_data(filename)      